# Instalar Librerias

In [ ]:
!pip install bs_ds           # bs_ds (Beautiful Soup Data Science)
!pip install fake_useragent  # fake_useragent se utiliza para generar encabezados de agente de usuario falsos (user agents) de manera aleatoria.
!pip install lxml            # lxml es una biblioteca de procesamiento de XML y HTML.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of bs-ds to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 8.5 MB/s eta 0:00:00
INFO: pip is 

#Librerias a utilizar

In [ ]:
# BeautifulSoup: Analiza HTML/XML para extraer datos.
from bs4 import BeautifulSoup
# requests: Realiza solicitudes HTTP para obtener contenido web.
import requests
# urljoin: Construye URLs completas a partir de enlaces.
from urllib.parse import urljoin
# se utiliza para guardar el archivo final en google drive
from google.colab import drive
drive.mount('/content/drive')
# libreria necesaria para crear un archivo CSV
import csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Realizar solicitud HTTP

In [ ]:
# URL a la que se accederá
url = 'https://www.informador.mx/seccion/ultimas-noticias'

# Envía una solicitud HTTP con tiempo de espera de 3 segundos
response = requests.get(url, timeout=3)

# Imprime el código de estado de la respuesta
print('Código de estado HTTP: ', response.status_code)

# Condicional que indica una conexión exitosa (200 OK) si no imprime un mensaje en consecuencia
if response.status_code == 200:
    print(f"{'--'*20}\n\tConexión exitosa OK.\n{'--'*20}\n")
else:
    print('Error.\n\n')

# Imprime el contenido de la respuesta de la solicitud
print(f"{'***'*20}\n\tRespuesta de la solicitud:\n{'***'*20}")

# Informacion de los encabezados de la respuesta HTTP recibida del servidor.
for k, v in response.headers.items():
    print(f"{k:{30}}: {v:{40}}")

Código de estado HTTP:  200
----------------------------------------
	Conexión exitosa OK.
----------------------------------------

************************************************************
	Respuesta de la solicitud:
************************************************************
Content-Type                  : text/html;charset=UTF-8                 
Transfer-Encoding             : chunked                                 
Connection                    : keep-alive                              
Date                          : Mon, 29 Apr 2024 16:10:14 GMT           
Access-Control-Allow-Origin   : *                                       
Last-Modified                 : Mon, 29 Apr 2024 16:10:14 GMT           
Content-Encoding              : gzip                                    
X-Cacheable                   : YES                                     
Cache-Control                 : max-age=300, public, stale-if-error=84600, stale-while-revalidate=120
X-Cacheable-TTL               :

In [ ]:
# Ruta al archivo CSV en tu Google Drive
file_path = '/content/drive/MyDrive/scrap/texto_extraido.csv'
# Realiza una solicitud HTTP GET a la URL
response = requests.get(url)
# Extrae el contenido HTML de la respuesta
content = response.text
# Crea un objeto BeautifulSoup para analizar el contenido HTML
soup = BeautifulSoup(content, 'lxml')

from datetime import datetime


# Encuentra titulos, fechas ,secciones de las Ultimas noticias.
# para esto es necesario inspeccionar la pagina y obtener las partes requeridas
t_encabezado = soup.find('h2', class_='mod-title')
news_titles = soup.find_all('h2', class_='news-title')
news_dates = soup.find_all('time', class_='news-date')
news_sections = soup.find_all('span', class_='news-section')

# Obtiene el texto del encabezado
texto_h2 = t_encabezado.text
print(f"{'---'*20}\n{texto_h2}\n{'---'*20}\n")


# Abrir el archivo CSV para escribir la cabecera si es la primera iteración
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Fecha', 'Seccion', 'Noticia', 'URL', 'Sinopsis', 'autor'])

# iterar entre las variables anteriores
for i, (news_date, news_section, news_title) in enumerate(zip(news_dates, news_sections, news_titles), start=1):

    # Muestra el contenido del atributo datetime de <time> (fecha)
    date_text = news_date['datetime']

    # Muestra el texto de <span> (seccion)
    section_text = news_section.get_text(strip=True)
    cleaned_text = ' '.join(section_text.split())
    #muestra el numero de iteraciones
    print(f"Numero Noticia {i} ")
    print(f"{'***'*20}\nSección: {cleaned_text}")

    # Extraer el enlace de redireccion q hay dentro del titulo

    # Encuentra el elemento <a> dentro del elemento h2 actual
    link_element = news_title.find('a')
    # Obtiene el enlace (href) y el texto del elemento <a>
    link_href = link_element.get('href')
    #muestra el texto de titulo de la noticia
    link_text = link_element.get_text(strip=True)

    #unir la url con el link del titulo de la noticia
    full_url = urljoin(url, link_href)

    #muestra resultados
    print(f"Noticia: {link_text}")
    print(f"Fecha: {date_text}")
    print(f"URL: {full_url}\n")

    #obtener el nombre autor q esta dentro del link de cada noticia
    # Realiza una solicitud HTTP a la URL completa
    linked_page_response = requests.get(full_url)

    # Crea un objeto BeautifulSoup para analizar el contenido de la página vinculada
    linked_page_soup = BeautifulSoup(linked_page_response.content, 'lxml')

    # Encuentra autores dentro de la pagina
    linked_page_content = linked_page_soup.find('p', class_='news-author')
    #linked_page_content_text = linked_page_content.get_text(strip=True).replace('por', '')

    # Imprime el contenido de la página vinculada
    print(f"Contenido del URL de la noticia:\n{linked_page_content.get_text(strip=True)}\n{'***' * 20}\n")

    #extra mostra primer parrafo de la noticia
    parrafo = linked_page_soup.find('div', class_='news-excerpt')

    # Imprime el contenido de la página vinculada
    print(f"{'▒▒▒'*20 }\n\tSinopsis:\n{parrafo.get_text(strip=True)}\n\n{'▒▒▒'* 20}\n\n")

    formatted_date = date_text.replace("T", " ")

# Convertir la cadena a un objeto datetime

    fecha_objeto = datetime.strptime(date_text, "%Y-%d-%mT%H:%M")


# Formatear la fecha y hora en otro formato ISO 8601
    fecha_iso8601_otro_formato = fecha_objeto.strftime("%Y-%m-%dT%H:%M:%S")
   # Escribe los datos en el archivo CSV
    with open(file_path, 'a', newline='') as file:
     writer = csv.writer(file)
     writer.writerow([fecha_objeto, cleaned_text, link_text, full_url, parrafo.get_text(strip=True), linked_page_content.get_text(strip=True)[4:]])